<div style="position: relative; width: 100%; height: 300px; display: flex; justify-content: center; align-items: center;">
    <img src="https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fbac2bfe0-2638-4d21-a44f-d51d5acd580f_1980x1320.jpeg" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; z-index: 0; opacity: 1.0; border-radius: 37px" >
    <div style="border-radius: 10px; position: relative; z-index: 1; text-align: center; background-color: rgba(50, 50, 50, 0.5); color: rgb(255, 150, 150); display: flex; flex-direction: column; align-items: center; text-align: center; justify-content: center; width: 100%; margin: 10%; padding: 5px ">
            <h1 style="text-align: center; width: 100%" >Data Analytics - NBA </h1>
    </div>
</div>

<h1 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgb(255, 219, 36); background-color: rgba(255, 73, 182, 0.5); padding: 30px; border-style: solid; border-radius: 10px;"> Imports & Load Data </h1>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

sns.set()
sns.set_palette('spring')
SNS_CMAP = 'spring'

colors = sns.palettes.color_palette(SNS_CMAP)

pd.options.mode.chained_assignment = None

In [21]:
df = pd.read_csv("nba.csv")

In [4]:
df.head().style.background_gradient(cmap=SNS_CMAP).apply(lambda s: ['background-color: rgba(255, 109, 146, 0.5)' if i % 2 == 0 else 'background-color: rgba(255, 182, 73, 0.5)' for i in range(len(s))])

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.000000,PG,25.000000,6-2,180.000000,Texas,7730337.000000
1,Jae Crowder,Boston Celtics,99.000000,SF,25.000000,6-6,235.000000,Marquette,6796117.000000
2,John Holland,Boston Celtics,30.000000,SG,27.000000,6-5,205.000000,Boston University,nan
3,R.J. Hunter,Boston Celtics,28.000000,SG,22.000000,6-5,185.000000,Georgia State,1148640.000000
4,Jonas Jerebko,Boston Celtics,8.000000,PF,29.000000,6-10,231.000000,nan,5000000.000000


In [5]:
df.describe().iloc[1:].style.background_gradient(cmap=SNS_CMAP)

,Number,Age,Weight,Salary
mean,17.678337,26.938731,221.522976,4842684.105381
std,15.966090,4.404016,26.368343,5229237.598910
min,0.000000,19.000000,161.000000,30888.000000
25%,5.000000,24.000000,200.000000,1044792.250000
50%,13.000000,26.000000,220.000000,2839073.000000
75%,25.000000,30.000000,240.000000,6500000.000000
max,99.000000,40.000000,307.000000,25000000.000000


In [6]:
agg_df = df.agg(["nunique", "unique", lambda x:x.isna().sum(), "dtypes"]).T
agg_df['unique'] = agg_df['unique'].apply(lambda x: x if len(x)<10 else x[:10])
agg_df.style.apply(lambda s: ['background-color: rgba(255, 109, 146, 0.5)' if i % 2 == 0 else 'background-color: rgba(255, 182, 73, 0.5)' for i in range(len(s))])

,nunique,unique,,dtypes
Name,457,['Avery Bradley' 'Jae Crowder' 'John Holland' 'R.J. Hunter' 'Jonas Jerebko' 'Amir Johnson' 'Jordan Mickey' 'Kelly Olynyk' 'Terry Rozier' 'Marcus Smart'],1,object
Team,30,['Boston Celtics' 'Brooklyn Nets' 'New York Knicks' 'Philadelphia 76ers' 'Toronto Raptors' 'Golden State Warriors' 'Los Angeles Clippers' 'Los Angeles Lakers' 'Phoenix Suns' 'Sacramento Kings'],1,object
Number,53,[ 0. 99. 30. 28. 8. 90. 55. 41. 12. 36.],1,float64
Position,5,['PG' 'SF' 'SG' 'PF' 'C' nan],1,object
Age,22,[25. 27. 22. 29. 21. 24. 20. 26. 28. 32.],1,float64
Height,18,['6-2' '6-6' '6-5' '6-10' '6-9' '6-8' '7-0' '6-4' '5-9' '6-7'],1,object
Weight,87,[180. 235. 205. 185. 231. 240. 238. 190. 220. 260.],1,float64
College,118,['Texas' 'Marquette' 'Boston University' 'Georgia State' nan 'LSU' 'Gonzaga' 'Louisville' 'Oklahoma State' 'Ohio State'],85,object
Salary,309,[ 7730337. 6796117. nan 1148640. 5000000. 12000000. 1170960. 2165160. 1824360. 3431040.],12,float64


<h1 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgb(255, 219, 36); background-color: rgba(255, 73, 182, 0.5); padding: 30px; border-style: solid; border-radius: 10px;"> Imputation </h1>

In [7]:
df_copy = df.copy()

<h3 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgba(255, 109, 146, 0.8); background-color: rgba(255, 182, 73, 0.3); padding: 10px; border-style: solid; border-radius: 10px;"> Drop missing Values </h3>

In [10]:
df = df_copy.copy()
df.dropna(axis=0, inplace=True)
df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.000000,PG,25.000000,6-2,180.000000,Texas,7730337.000000
1,Jae Crowder,Boston Celtics,99.000000,SF,25.000000,6-6,235.000000,Marquette,6796117.000000
3,R.J. Hunter,Boston Celtics,28.000000,SG,22.000000,6-5,185.000000,Georgia State,1148640.000000
6,Jordan Mickey,Boston Celtics,55.000000,PF,21.000000,6-8,235.000000,LSU,1170960.000000
7,Kelly Olynyk,Boston Celtics,41.000000,C,25.000000,7-0,238.000000,Gonzaga,2165160.000000


<h3 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgba(255, 109, 146, 0.8); background-color: rgba(255, 182, 73, 0.3); padding: 10px; border-style: solid; border-radius: 10px;"> Imputation with constant value </h3>

In [11]:
df = df_copy.copy()
df.fillna(0, inplace=True)
df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.000000,PG,25.000000,6-2,180.000000,Texas,7730337.000000
1,Jae Crowder,Boston Celtics,99.000000,SF,25.000000,6-6,235.000000,Marquette,6796117.000000
2,John Holland,Boston Celtics,30.000000,SG,27.000000,6-5,205.000000,Boston University,0.000000
3,R.J. Hunter,Boston Celtics,28.000000,SG,22.000000,6-5,185.000000,Georgia State,1148640.000000
4,Jonas Jerebko,Boston Celtics,8.000000,PF,29.000000,6-10,231.000000,0,5000000.000000


<h3 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgba(255, 109, 146, 0.8); background-color: rgba(255, 182, 73, 0.3); padding: 10px; border-style: solid; border-radius: 10px;"> Imputation with Central Measure </h3>

In [12]:
df = df_copy.copy()
numerics = ["int", "int32", "int64", "float", "float32", "float64"]
numeric_cols = df.select_dtypes(numerics).columns
cat_cols =  df.select_dtypes(exclude=numerics).columns

In [13]:
for col in numeric_cols:
    mean_val = df[col].mean().astype(int)
    df[col].fillna(mean_val, inplace=True)
for col in cat_cols:
    mode_val = df[col].mode()[0]
    df[col].fillna(mode_val, inplace=True)

In [16]:
df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.000000,PG,25.000000,6-2,180.000000,Texas,7730337.000000
1,Jae Crowder,Boston Celtics,99.000000,SF,25.000000,6-6,235.000000,Marquette,6796117.000000
2,John Holland,Boston Celtics,30.000000,SG,27.000000,6-5,205.000000,Boston University,4842684.000000
3,R.J. Hunter,Boston Celtics,28.000000,SG,22.000000,6-5,185.000000,Georgia State,1148640.000000
4,Jonas Jerebko,Boston Celtics,8.000000,PF,29.000000,6-10,231.000000,Kentucky,5000000.000000


<h3 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgba(255, 109, 146, 0.8); background-color: rgba(255, 182, 73, 0.3); padding: 10px; border-style: solid; border-radius: 10px;"> Imputation with sklearn </h3>

In [17]:
from sklearn.impute import KNNImputer

df = df_copy.copy()

imputer = KNNImputer()
imputer.fit(df[numeric_cols])

KNNImputer()

In [18]:
df[numeric_cols] = imputer.transform(df[numeric_cols])

In [20]:
df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.000000,PG,25.000000,6-2,180.000000,Texas,7730337.000000
1,Jae Crowder,Boston Celtics,99.000000,SF,25.000000,6-6,235.000000,Marquette,6796117.000000
2,John Holland,Boston Celtics,30.000000,SG,27.000000,6-5,205.000000,Boston University,1301333.800000
3,R.J. Hunter,Boston Celtics,28.000000,SG,22.000000,6-5,185.000000,Georgia State,1148640.000000
4,Jonas Jerebko,Boston Celtics,8.000000,PF,29.000000,6-10,231.000000,nan,5000000.000000


<h1 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgb(255, 219, 36); background-color: rgba(255, 73, 182, 0.5); padding: 30px; border-style: solid; border-radius: 10px;">Summary Statistics </h1>

<h3 style="text-align: center; font-family: 'Roboto', sans-serif; color: rgba(255, 109, 146, 0.8); background-color: rgba(255, 182, 73, 0.3); padding: 10px; border-style: solid; border-radius: 10px;"> Position </h3>

In [57]:
df.groupby("Position")["Salary"].median()

Position
C     3811060.0
PF    3000000.0
PG    3069583.5
SF    2041080.0
SG    1938840.0
Name: Salary, dtype: float64

In [60]:
df.groupby("Position")["Salary"].mean().astype(int)

Position
C     5967052
PF    4588432
PG    5131253
SF    4833896
SG    3957493
Name: Salary, dtype: int32

In [66]:
 df.groupby("Position")["Salary"].apply(lambda x: x.mode())

Position   
C         0    947276.0
PF        0    845059.0
          1    947276.0
PG        0    947276.0
SF        0    845059.0
SG        0    947276.0
Name: Salary, dtype: float64

In [69]:
 df.groupby("Position")["Salary"].apply(lambda x: x.mode()).groupby("Position").first()

Position
C     947276.0
PF    845059.0
PG    947276.0
SF    845059.0
SG    947276.0
Name: Salary, dtype: float64

In [23]:
 df.groupby("Position")["Salary"].var()

Position
C     3.350082e+13
PF    2.304052e+13
PG    2.552078e+13
SF    3.614281e+13
SG    2.017455e+13
Name: Salary, dtype: float64

In [24]:
 df.groupby("Position")["Salary"].std()

Position
C     5.787989e+06
PF    4.800054e+06
PG    5.051809e+06
SF    6.011889e+06
SG    4.491609e+06
Name: Salary, dtype: float64